##  概念

正则化是一种旨在减少泛化误差而不是训练误差的方法，是机器学习中非常重要并且有效的技术，特别是在深度学习模型中，由于其模型参数非常多非常容易产生过拟合。因此需要引入正则化来抑制模型过拟合现象，提高模型泛化能力，常用的正则化方法有：  

1 参数添加约束，例如L1、L2范数等  

2 训练集合扩充，例如添加噪声、数据变换等


## 1. L2 正则化  

L2 正则化是最常用的正则化方法。可以通过惩罚目标函数中所有参数的平方将其实现。即对于网络中的每个权重 w，向目标函数中增加一个 $\frac{1}{2}\lambda w^2$，其中 $\lambda$ 是正则化强度。前面这个 $\frac{1}{2}$ 很常见，是因为加上 $\frac{1}{2}$ 后，该式子关于 w 梯度就是 $\lambda w$ 而不是 $2\lambda w$ 了。L2 正则化可以直观理解为它对于大数值的权重向量进行严厉惩罚，倾向于更加分散的权重向量,这样就有了一个优良的特性：使网络更倾向于使用所有输入特征，而不是严重依赖输入特征中某些小部分特征。最后需要注意在梯度下降和参数更新的时候，使用 L2 正则化意味着所有的权重都以 $w += -\lambda * W$ 向着 0 线性下降。

## 2. L1正则化  

L1 正则化是另一个相对常用的正则化方法。对于每个 w 我们都向目标函数增加一个$\lambda|w|$。L1 和 L2 正则化也可以进行组合：
$\lambda_1|w|+\lambda_2w^2$，这也被称作 Elastic net regularizaton。L1 正则化有一个有趣的性质，它会让权重向量在最优化的过程中变得稀疏（即非常接近0）。也就是说，使用 L1 正则化的神经元最后使用的是它们最重要的输入数据的稀疏子集，同时对于噪音输入则几乎是不变的了。相较 L1 正则化，L2 正则化中的权重向量大多是分散的小数字。在实践中，如果不是特别关注某些明确的特征选择，一般说来 L2 正则化都会比 L1 正则化效果好。

L2 和 L1 正则具有不同的性质：L2得到平滑的权值， L1得到稀疏的权值  

当几个特征的共线性很高，并且对分类都很重要，L2 倾向于将权值平分给这些特征，从而使得这些有用的特征都得以留下；L1 则会随机选择其中一个特征，而扔掉其它的特征，所以说L2平滑，L1稀疏。下面从两个方面解释这两个性质。  

### 2.1 数学推导  

首先来看 L2 和 L1 的梯度的更新方式：

$L2 = \frac {1}{2}(w_1^2 + w_2^2 + ... + w_n^2), \ \ \ \ \frac {\partial L_2}{w_i} = w_i$  

$L1 = |w_1| + |w_2| + ... + |w_n| , \ \ \ \ \frac {\partial L_1}{w_i} = sign(w_i) = 1 \ or \  -1$  

L2 的权值更新公式为 $w_i = w_i - \eta * w_i$，可见权值不断变小，但是因为每次都减少上一次的 $\eta$ 倍，在小数值时候，权值下降的较慢，很难下降到 0，会收敛到较小但不为 0 的值，所以训练好的权值比较平滑。


L1 的权值更新公式为 $w_i = w_i - \eta * sign(w_i)$，也就是说权值每次更新都固定减少一个特定的值 $\eta$，在小数值时，权值下降较快，经过若干次迭代之后，权值有可能减少到 0。  

### 2.2 几何空间  

下图是 PRML 中 介绍 L2 与 L1 不同性质的图：  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/regularization/1.jpg?raw=true)

这里刚开始看不明白，优化的目标函数应该是 $\min\limits_w E_D(w) + \lambda E_R(w)$, 损失函数包含了两部分，应该结合起来进行优化，但图里面将两项分开进行优化了，这里进行简单的解释：  

$\min\limits_w E_D(w) + \lambda E_R(w)$ 和 $\begin{cases} \min\limits_w E_D(w) \\ s.t.E_R(w) \leq \eta \end{cases}$ 是等价的，即对一个特定的 $\lambda$ 总存在一个 $\eta$ 使得这两个问题是等价的。

上图表达的是对目标函数的等价变换之后的形式，把 w 的解限制在黄色区域内，同时使得经验损失尽可能小。  

为了便于可视化，上图考虑两维的情况，在(w1,w2)平面上可以画出目标函数的等高线，而约束条件则成为平面上半径为 $\eta$ 的一个 norm ball。等高线与 norm ball 首次相交的地方就是最优解：  
 
不同的同心圆就是优化过程中遇到的等高线，可以看到，L1-ball 与 L2-ball 的不同就在于 L1 在和每个坐标轴相交的地方都有“角”出现，除非目标函数比较特殊(位于特定的位置)，否则大部分条件下都会在角的地方相交，角的位置就会产生稀疏性。

相比之下，L2-ball 就没有这样的性质，因为没有角，所以第一次相交的地方出现在具有稀疏性的位置的概率就变得非常小了，所以 L2 计算出来的权重就比较平滑。

## 3. Dropout  

随机失活（Dropout）是一个简单又极其有效的正则化方法。该方法由 Srivastava 在论文 Dropout: A Simple Way to Prevent Neural Networks from Overfitting 中提出的，与 L1 正则化，L2 正则化等方法互为补充。在训练的时候，随机失活的实现方法是让神经元以超参数 p 的概率被激活或者被设置为0。  

在训练过程中，随机失活可以被认为是对完整的神经网络抽样出一些子集，每次基于输入数据只更新子网络的参数。在测试过程中不使用随机失活，可以理解为是对数量巨大的子网络们做了模型集成，以此来计算出一个平均的预测。  

理论上 Dropout 在卷积层和 FC（全连接）层都能使用。在 Hinton 组关于 Imagenet 的实验中，只在最后的两个 FC 层用了 dropout。但是很少对卷积层采用 dropout，也尽量不在太多的层采用 dropout。 

In [ ]:
def dropout_forward(x, dropout_param):
  """
  Performs the forward pass for (inverted) dropout.
  Inputs:
  - x: Input data, of any shape
  - dropout_param: A dictionary with the following keys:
    - p: Dropout parameter. We drop each neuron output with probability p.
    - mode: 'test' or 'train'. If the mode is train, then perform dropout;
      if the mode is test, then just return the input.
    - seed: Seed for the random number generator. Passing seed makes this
      function deterministic, which is needed for gradient checking but not in
      real networks.

  Outputs:
  - out: Array of the same shape as x.
  - cache: A tuple (dropout_param, mask). In training mode, mask is the dropout
    mask that was used to multiply the input; in test mode, mask is None.
  """
  p, mode = dropout_param['p'], dropout_param['mode']
  if 'seed' in dropout_param:
    np.random.seed(dropout_param['seed'])

  mask = None
  out = None

  if mode == 'train':
    mask = (np.random.rand(*x.shape) >= p) / (1 - p)
    out = x * mask
  elif mode == 'test':
    out = x

  cache = (dropout_param, mask)
  out = out.astype(x.dtype, copy=False)

  return out, cache

def dropout_backward(dout, cache):
  """
  Perform the backward pass for (inverted) dropout.
  Inputs:
  - dout: Upstream derivatives, of any shape
  - cache: (dropout_param, mask) from dropout_forward.
  """
  dropout_param, mask = cache
  mode = dropout_param['mode']
  
  dx = None
  if mode == 'train':
    dx = dout * mask
  elif mode == 'test':
    dx = dout
  return dx

上面是 dropout 层的前向和反向过程，dropout 的前向主要有两个过程：  

1、让某个神经元以概率 p 停止工作，其实就是让它的激活值以概率 p 变为 0。假设某一层网络有 1000 个神经元，p = 0.5，那么这一层神经元经过 dropout 后，这一层大概会有 500个 的神经元的激活值被置为 0。  

2、经过上面屏蔽掉某些神经元，使其激活值为 0 以后，还需要对神经元激活值进行 rescale，也就是乘以 1/(1-p)。  

在训练时将 dropout 之后的神经元激活值进行 rescale 的过程被称为 inverted dropout。当模型使用了dropout layer，训练的时候只有占比为 1-p 的隐藏层单元参与训练，那么在预测的时候，如果所有的隐藏层单元都需要参与进来，则得到的结果相比训练时平均要大 1/(1-p)，为了避免这种情况，就需要测试的时候将输出结果乘以 1-p,使下一层的输入规模保持不变。

而利用 inverted dropout，可以在训练的时候直接将 dropout 后留下的权重扩大 1/(1-p) 倍，这样就可以使结果的 scale 保持不变，而在预测的时候也不用做额外的操作了，更方便一些。

## 4. DropConnect  

DropConnect 来源于《Regularization of Neural Networks using DropConnect》这篇文章。  

图片2

DropConnect 是 Hinton Dropout的泛化版本，用于规范神经网络中的大型全连接层。在使用 Dropout 进行训练时，每个图层中随机选择的激活子集被设置为零，DropConnect 将网络中随机选择的权重子集设置为零。Dropout 是将输出随机置0，DropConnect是将权重随机置 0，一个是对输出进行操作，一个是对输入进行操作，理论上 DropConnect 比 Dropout 随机的更加充分。  

作者做了一些对比试验，发现效果并不比 Dropout 优秀太多，反而计算量要大很多，因此到目前 DropConnect 并没有得到广泛的应用。

## 5. DropNeuron  
 
DropNeuron 来源于 DropNeuron: Simplifying the Structure of Deep Neural Networks 这篇论文。通过将大型网络中与特定神经元有关的权重全变为 0，实现对该神经元的 Dropout，从而可以在大型网络中学习出一个小的网络，达到正则化的效果。  

该文章想要解决的问题是在训练过程中简化神经网络，在可接受的计算时间内实现合理的性能，提出了一种通过网络架构正则化来优化深度神经网络的新方法。DropNeuron 旨在从大型随机初始化模型中训练一个小模型，而不是压缩或减小大模型。DropNeuron 可以与其他正则化技术混合使用，如 dropout，L1，L2 等。  

图片3

通过设计合适的损失函数，将所有传入特定神经元连接的权重和所有传出神经元连接的权重调整为零，就可以实现对特定神经元的 Drop。


如图所示，从$l-1$神经元 到 $l$ 层神经元之间的连接权重 $W^l = [(W_{1,:}^l),...,(W_{n^{l-1},:})]^T = [W_{:,1}^l,...,W_{:,n^l}^l]$  

$W_{i,:}^l$ 是 权重矩阵 $W^l$ 的第 i 行，表示了从$l-1$神经元 到 $l$ 层神经元的输入权重，$W_{:,j}^l$ 是 权重矩阵 $W^l$ 的第 j 列，表示了从$l$神经元 到 $l+1$ 层神经元的输出权重。

DropNeuron 引入了两个正则化因子：  

$li\_regulariser := \lambda_{l_i} \sum_{l=1}^L \sum_{j=1}^{n^l} {\mid\mid} W_{:,j}^l {\mid\mid}_2 $，这个因子用来对每层神经元的输入权重进行正则。  

$lo\_regulariser := \lambda_{l_o} \sum_{l=1}^L \sum_{i=1}^{n^{l-1}} {\mid\mid} W_{i,:}^l {\mid\mid}_2 $，这个因子用来对每层神经元的输出权重进行正则。 

通过在损失函数中加入上面两个正则化因子，可以实现 DropNeuron。

## 6. Maxout  

DropNeuron 来源于 Maxout Networks 这篇论文。  

Maxout是深度学习网络中的一层网络，就像池化层、卷积层一样等，可以把 maxout 看成是网络的激活函数层也可以看做是正则化的一种，假设网络某一层的输入特征向量为：$X = (x_1,x_2,……,x_d)$，也就是我们输入是d个神经元。Maxout隐藏层每个神经元的计算公式如下：  

$ f(x) = max(x^T*W_{ij} + b_{ij}) $  

可以这么理解，本来传统的MLP算法在第 i 层到第 i+1 层，参数只有一组，然而现在我们不这么做，我们在这一层同时训练 n 组参数，然后选择激活值最大的作为下一层神经元的激活值。

## 7. DisturbLabel  

DropNeuron 来源于 DisturbLabel: Regularizing CNN on the Loss Layer 这篇论文。提出了一个简单有效的防止 CNN 模型过拟合（或者说正则化）的方法：在每个 mini-batch 内，随机将一部分样本的 Groundtruth 替换为随机类别。和 Dropout 配合使用，效果更佳。

图片4  

图片5

## 8.  Early stopping  

Early stopping 是一种迭代次数截断的方法来防止过拟合的方法，即在模型对训练数据集迭代收敛之前停止迭代来防止过拟合。  

具体方法：  

在每个 Epoch 结束时(一个 Epoch 对所有的训练集完成一轮遍历) 计算验证集的准确率，当准确率不再提高时，停止训练。一个重点是怎样才认为验证集的准确率不再提高？并不是说验证集的准确率一降下来便认为不再提高了，因为可能经过这个 Epoch 后，准确率降低了，但随后的 Epoch 准确率又上去了，所以不能根据一两次的连续降低就判断不再提高。一般的做法是，在训练的过程中，记录到目前为止最好的验证集准确率，当连续 n (10）次没达到最佳的准确率时，则可以认为准确率不再提高此时便可以停止迭代了（Early Stopping）。

## 参考  

1 [CS231n官方笔记授权翻译总集](https://zhuanlan.zhihu.com/p/21930884?refer=intelligentunit)  
2 [神经网络Dropout层中为什么dropout后还需要进行rescale？](https://www.zhihu.com/question/61751133)  